In [1]:
from dynamixel_sdk import *
from simple_pid import PID

# Control table address
ADDR_MX_TORQUE_ENABLE      = 24               # Control table address is different in Dynamixel model
ADDR_MX_GOAL_POSITION      = 30
ADDR_MX_PRESENT_POSITION   = 36
ADDR_MX_MOVING_SPEED       = 32
ADDR_MX_PRESENT_SPEED      = 38

# Protocol version
PROTOCOL_VERSION            = 1.0               # See which protocol version is used in the Dynamixel

# Default setting
DXL_ID_1                      = 1                 # Dynamixel ID : 1
DXL_ID_2                      = 6                 
DXL_ID_3                      = 2                 
DXL_ID_4                      = 3                 
DXL_ID_5                      = 4                 
BAUDRATE                      = 1000000             # Dynamixel default baudrate : 57600
DEVICE_PORT                   = "COM6"          # Check which port is being used on your controller

TORQUE_ENABLE               = 1                 # Value for enabling the torque
TORQUE_DISABLE              = 0                 # Value for disabling the torque
MAX_SPEED                   = 20

# PID Parameters
KP = 0
KI = 0.1
KD = 0
SAMPLE_TIME = 0.01
UPPER_LIMIT = 200
LOWER_LIMIT = -100


# Create PID 
pid = [PID() for x in range(2)]

for x in range(2):
    pid[x].sample_time = SAMPLE_TIME
    pid[x].tunings = (KP, KI, KD)
    pid[x].output_limit = (LOWER_LIMIT, UPPER_LIMIT)
    pid[x].setpoint = 512
    pid[x].auto_mode = False

# Create handler objects
portHandler = PortHandler(DEVICE_PORT)
packetHandler = PacketHandler(PROTOCOL_VERSION)

# Open port and set baudrate
portHandler.openPort()
portHandler.setBaudRate(BAUDRATE)


# Enable Dynamixel
dxl_comm_result, dxl_error = packetHandler.write1ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_TORQUE_ENABLE, TORQUE_ENABLE)
dxl_comm_result, dxl_error = packetHandler.write1ByteTxRx(portHandler, DXL_ID_3, ADDR_MX_TORQUE_ENABLE, TORQUE_ENABLE)




AttributeError: 'list' object has no attribute 'sample_time'

In [158]:
# Limit speed
#packetHandler.write4ByteTxOnly(portHandler, DXL_ID_2, ADDR_MX_MOVING_SPEED, 20)
dxl_comm_result, dxl_error = packetHandler.write2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_MOVING_SPEED, 20)


In [159]:
speed, dxl_comm_result, dxl_error = packetHandler.read2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_MOVING_SPEED)
speed

20

In [160]:
# Set to vertical 
dxl_comm_result, dxl_error = packetHandler.write2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_GOAL_POSITION, 512)

In [164]:
currentpos, dxl_comm_result, dxl_error = packetHandler.read2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_PRESENT_POSITION)
print(currentpos)
print(pid.setpoint)

511
512


In [165]:
# Enable PID to ensure vertical
pid.auto_mode = True

# Get current speed 
speed, dxl_comm_result, dxl_error = packetHandler.read2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_PRESENT_SPEED)
if speed < 2:
    while True:
        # Get current position
        currentpos, dxl_comm_result, dxl_error = packetHandler.read2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_PRESENT_POSITION)
        output = int(pid(currentpos)) + 512
        print(output)
    
        # Output PID to motor
        dxl_comm_result, dxl_error = packetHandler.write2ByteTxRx(portHandler, DXL_ID_2, ADDR_MX_GOAL_POSITION, output)
    
        
        
        if abs(currentpos - 512) < 1:
            break

519
519
519
519


In [131]:
output

6566

In [147]:
portHandler.closePort()